# OR case 1

In [9]:
import pandas as pd
import openpyxl
from gurobipy import *
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px
import plotly.io as pio

In [10]:
pio.renderers
pio.renderers.default = "notebook_connected"

# display settings
from IPython.display import display
pd.options.display.max_columns = None
pd.options.display.max_rows = None
pd.set_option('display.float_format', lambda x: '%.4f' % x)

# 第二、三題的甘特圖
def gantt_plot_2_3(x_opt, P, y_opt, problem_no, open_hour=0, open_min=0):
    import datetime

    I = len(x_opt)
    S = len(x_opt[0])
    M = len(y_opt[0][0])
    schedule_df = pd.DataFrame()
    open_time = datetime.datetime(2000, 1, 1, hour=open_hour, minute=open_min)
    schedule_df['START_TIME'] = [(open_time + datetime.timedelta(hours=x_opt[i][s]) - datetime.timedelta(hours=P[i][s])) for i in range(I) for s in range(S)]
    schedule_df['END_TIME'] = [(open_time + datetime.timedelta(hours=x_opt[i][s])) for i in range(I) for s in range(S)]

    machine_ID = []
    for i in range(I):
        for s in range(S):
            for m in range(M):
                if y_opt[i][s][m] == 1:
                    machine_ID.append(str(m + 1))
    schedule_df['Machine_ID'] = machine_ID

    schedule_df['Job_ID'] = [str(i + 1) for i in range(I) for s in range(S)]
    color_list = ["#C0392B", "#EC7063", "#1F618D", "#34495E", "#3498DB", "#1E8449", "#28B463", "#F1C40F", "#F39C12", "#A6ACAF", "#7D3C98", "#9B59B6", "#2C3E50", "#EF553B", "#636EFA", "#BDC3C7", "#F39C12"]
    fig = px.timeline(schedule_df, x_start="START_TIME", x_end="END_TIME", y="Machine_ID", color="Job_ID",
                      color_discrete_sequence=color_list[0:I],
                      category_orders={"Job_ID": [str(i+1) for i in range(I)], "Machine_ID": [str(m+1) for m in range(M)]},
                      title="Instance " + str(1) + " Problem " + str(problem_no) + " Gantt Chart")
    fig.update_xaxes(tickformat="%H:%M")
    fig.update_layout(legend_title="Lot ID")
    fig.show()
    fig.write_image("Instance " + str(1) + " Problem " + str(problem_no) + ".pdf")

In [11]:
# which test set
i = 0

root = '../tests/test'
subfileName = ".csv"

tardyamount2_2_2 = 0
path = root+ str(i) + subfileName
print ('path:', path)

path: ../tests/test0.csv


In [12]:
from huristic1_1_2 import huristic1_1_2

makespan0, tardyamount0 = huristic1_1_2(path)

print('tardyamount', tardyamount0)
print('makespan', makespan0)

tardyamount 0
makespan 14.658691490160027


In [13]:
from huristicPaper import huristicPaper

makespan, tardyamount, resultList = huristicPaper(path)

print('tardyamount', tardyamount)
print('makespan', makespan)

tardyamount 0
makespan 14.658691490160027


### 3.

### Constant Data

In [14]:
root1 = '../tests/test'
subfileName1 = ".csv"
path = root1+ str(i) + subfileName1
df = pd.read_csv(path)
df

,Unnamed: 0,Job ID,Stage-1 Processing Time,Stage-2 Processing Time,Stage-1 Machines,Stage-2 Machines,Due Time
0,0,0,1.2896,0.5107,"2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19","1,7,13,17",10.0029
1,1,1,0.8846,0.2977,"2,3,4,5,6,7,8,9,11,17,18,19","1,2,4,8,9,13,14",18.4496
2,2,2,0.8283,3.8450,"3,5,6,7,10,11,12,13,14,17","1,2,3,4,5,6,7,8,9,11,12,14,16,17,18,19",10.1672
3,3,3,2.4786,4.6428,"2,3,5,6,8,9,11,14,16,17","2,5,8,9,10,11,14",12.5758
4,4,4,1.6269,9.5415,"1,3,4,6,7,8,10,11,13,14,16,17,18",16,23.2470
5,5,5,5.9996,8.6591,"2,3,10,11","1,2,3,4,6,7,8,9,10,11,12,13,14,15,16,17,18,19",21.4370
6,6,6,4.7996,5.8106,"3,4,5,6,7,13","4,13,18,19",13.4846
7,7,7,2.3071,3.5087,"1,7,9,10,11,12,14,16,17,18,19","1,3,4,5,6,7,8,9,11,13,14,15,16,17,18,19",9.9473
8,8,8,2.6076,8.7832,18,"4,5,6,7,10,11,15,17,19",13.5081


In [15]:
# df['Stage-1 Machines']
# df['Stage-2 Machines']

In [16]:
max_amount = 0
for index, row in df.iterrows():
    x1 = row['Stage-1 Machines'].split(',')
    for element in x1:
        num = int(element)
        if(num > max_amount):
            max_amount = num
            
    temp = row['Stage-2 Machines']
    if(pd.isna(temp) == False):
        x2 = temp.split(',')
        for element in x2:
            num = int(element)
            if(num > max_amount):
                max_amount = num

In [17]:
max_amount

19

In [18]:
# instance i 中 job 的總數量
I = len(df) # order num
S = 2 # stage num


orders = range(0, I)
machines = range(0, max_amount)
stages = range(0, S)

In [19]:
P = []
for index, row in df.iterrows():
    P.append([])
    P[index].append(row['Stage-1 Processing Time'])
    P[index].append(row['Stage-2 Processing Time'])

In [20]:
P

[[1.2896055920390104, 0.5106820365657239],
 [0.8846096798429537, 0.2976714662447322],
 [0.8282584881461996, 3.845033381674608],
 [2.478641655940077, 4.6428150743957],
 [1.6268668826890764, 9.541542161515],
 [5.999631582262968, 8.65905990789706],
 [4.7996116991740125, 5.810612345832659],
 [2.3070976199463344, 3.5086918544120307],
 [2.607604645283049, 8.78316755323287]]

In [21]:
# C_ijm -> init 3 dim 都是 false 的東西
C =  []
for i in range(I):
    C.append([])
    C[i].append([0 for i in range(max_amount)])
    C[i].append([0 for i in range(max_amount)])

In [22]:
for index, row in df.iterrows():
    x1 = row['Stage-1 Machines'].split(',')
    for element in x1:
        num = int(element)
        C[index][0][num-1] = 1

        temp = row['Stage-2 Machines']
        if(pd.isna(temp) == False):
            x2 = temp.split(',')
            for element in x2:
                num = int(element)
                C[index][1][num-1] = 1

In [23]:
C

[[[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
  [1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0]],
 [[0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1],
  [1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0]],
 [[0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0],
  [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1]],
 [[0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0],
  [0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0]],
 [[1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]],
 [[0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
  [1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],
 [[0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1]],
 [[1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1],
  [1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1]],
 [[0, 0, 0, 0, 0

In [24]:
D = []
for index, row in df.iterrows():
    D.append(row['Due Time'])
D

[10.002877467903849,
 18.44959110542398,
 10.16718186695907,
 12.57583850931542,
 23.246963410667853,
 21.43700130894092,
 13.484570010111124,
 9.947349837986,
 13.508132567661516]

### Decision variable – Delay

In [25]:
Q3_delay = Model('Q3-delay')
Q3_delay.setParam('TimeLimit',100)

# x_is is the finish time of order i at stage s
x = {}
for i in orders:
    x[i] = {}
    for s in stages:
        x[i][s] = Q3_delay.addVar(lb = 0, vtype = GRB.CONTINUOUS, name = "x_" + str(i) + "," + str(s))

# y_ism = 1 if order i process on machine m  at stage s otherwise 0
y = {}
for i in orders:
    y[i] = {}
    for s in stages:
        y[i][s] = {}
        for m in machines:
            y[i][s][m] = Q3_delay.addVar(lb = 0, vtype = GRB.BINARY, name = "y_" + str(i) + "," + str(s) + "," + str(m))
        
# z_isjt = 1 if order i's stage s process before order j's stage t and they both process on the same machine otherwise 0
z = {}
for i in orders:
    z[i] = {}
    for s in stages:
        z[i][s] = {}
        for j in orders:
            z[i][s][j] = {}
            for t in stages:
                z[i][s][j][t] = Q3_delay.addVar(lb = 0, vtype = GRB.BINARY, name = "z_" + str(i) + "," + str(s) + "," + str(j) + "," + str(t))
        
# l_i = 1 if order i is late otherwise 0
l = {}
for i in orders:
    l[i] = Q3_delay.addVar(lb = 0, vtype = GRB.BINARY, name = "l_" + str(i))

Q3_delay.update()

Set parameter Username
Academic license - for non-commercial use only - expires 2023-04-27
Set parameter TimeLimit to value 100


### Setting the objective function

In [26]:
Q3_delay.setObjective(quicksum(l[i] for i in orders), GRB.MINIMIZE)

### Add constraints

In [27]:
Q3_delay.addConstrs((x[j][t] + P[i][s] - x[i][s] <= (quicksum(P[i][s] for i in orders for s in stages)+P[i][s]) * (z[i][s][j][t] + 2-y[i][s][m]-y[j][t][m]) for i in orders for j in range(i+1, I) for m in machines for s in stages for t in stages), 'constraints for x_is, z_isjt')
Q3_delay.addConstrs((x[i][s] + P[j][t] - x[j][t] <= (quicksum(P[i][s] for i in orders for s in stages)+P[j][t]) * (1-z[i][s][j][t] + 2-y[i][s][m]-y[j][t][m]) for i in orders for j in range(i+1, I) for m in machines for s in stages for t in stages), 'constraints for x_is, z_isjt')
Q3_delay.addConstrs((x[i][1] <= D[i] + quicksum(P[i][s] for s in stages for i in orders) * l[i] for i in orders), 'constraints for l_i')
Q3_delay.addConstrs((quicksum(y[i][s][m] for m in machines) == 1 for i in orders for s in stages), 'allocate orders to machines')
Q3_delay.addConstrs((x[i][s] >= P[i][s] for i in orders for s in stages), 'minimum x_i')
Q3_delay.addConstrs((x[i][1] >= x[i][0] + P[i][1] for i in orders), 'stage 2 should start after stage 1 is finished')

# C[i][s][m]
Q3_delay.addConstrs((y[i][s][m] <= (1 if C[i][s][m] == 0 else 1) for i in orders for s in stages for m in machines), 'constraints for cook machines')

Q3_delay.update()

In [28]:
Q3_delay.optimize()

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (mac64[rosetta2])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 5868 rows, 693 columns and 28098 nonzeros
Model fingerprint: 0x3d3e181e
Variable types: 18 continuous, 675 integer (675 binary)
Coefficient statistics:
  Matrix range     [1e+00, 8e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-01, 2e+02]
Found heuristic solution: objective 0.0000000

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 1: 0 

Optimal solution found (tolerance 1.00e-04)
Best objective 0.000000000000e+00, best bound 0.000000000000e+00, gap 0.0000%


In [29]:
a = Q3_delay.objVal
print("objective value =", a)

objective value = 0.0


### Decision variable – makespan

In [30]:
Q3_makespan = Model('Q3-makespan')
Q3_makespan.setParam('TimeLimit',100)

# x_is is the finish time of order i at stage s
x = {}
for i in orders:
    x[i] = {}
    for s in stages:
        x[i][s] = Q3_makespan.addVar(lb = 0, vtype = GRB.CONTINUOUS, name = "x_" + str(i) + "," + str(s))

# y_ism = 1 if order i process on machine m  at stage s otherwise 0
y = {}
for i in orders:
    y[i] = {}
    for s in stages:
        y[i][s] = {}
        for m in machines:
            y[i][s][m] = Q3_makespan.addVar(lb = 0, vtype = GRB.BINARY, name = "y_" + str(i) + "," + str(s) + "," + str(m))
        
# z_isjt = 1 if order i's stage s process before order j's stage t and they both process on the same machine otherwise 0
z = {}
for i in orders:
    z[i] = {}
    for s in stages:
        z[i][s] = {}
        for j in orders:
            z[i][s][j] = {}
            for t in stages:
                z[i][s][j][t] = Q3_makespan.addVar(lb = 0, vtype = GRB.BINARY, name = "z_" + str(i) + "," + str(s) + "," + str(j) + "," + str(t))
        
# l_i = 1 if order i is late otherwise 0
l = {}
for i in orders:
    l[i] = Q3_makespan.addVar(lb = 0, vtype = GRB.BINARY, name = "l_" + str(i))

# w is the makespan of the solution
w = Q3_makespan.addVar(lb = 0, vtype = GRB.CONTINUOUS, name = "w")

Q3_makespan.update()

Set parameter TimeLimit to value 100


### Setting the objective function

In [31]:
Q3_makespan.setObjective(w, GRB.MINIMIZE)

### Add constraints

In [32]:
Q3_makespan.addConstrs((x[j][t] + P[i][s] - x[i][s] <= (quicksum(P[i][s] for i in orders for s in stages)+P[i][s]) * (z[i][s][j][t] + 2-y[i][s][m]-y[j][t][m]) for i in orders for j in range(i+1, I) for m in machines for s in stages for t in stages), 'constraints for x_is, z_isjt')
Q3_makespan.addConstrs((x[i][s] + P[j][t] - x[j][t] <= (quicksum(P[i][s] for i in orders for s in stages)+P[j][t]) * (1-z[i][s][j][t] + 2-y[i][s][m]-y[j][t][m]) for i in orders for j in range(i+1, I) for m in machines for s in stages for t in stages), 'constraints for x_is, z_isjt')
Q3_makespan.addConstrs((x[i][1] <= D[i] + quicksum(P[i][s] for s in stages for i in orders) * l[i] for i in orders), 'constraints for l_i')
Q3_makespan.addConstrs((quicksum(y[i][s][m] for m in machines) == 1 for i in orders for s in stages), 'allocate orders to machines')
Q3_makespan.addConstrs((x[i][s] >= P[i][s] for i in orders for s in stages), 'minimum x_i')
Q3_makespan.addConstrs((x[i][1] >= x[i][0] + P[i][1] for i in orders), 'stage 2 should start after stage 1 is finished')
# C_ism
Q3_makespan.addConstrs((y[i][s][m] <= (1 if C[i][s][m] == 0 else 1) for i in orders for s in stages for m in machines), 'constraints for cook machines')
Q3_makespan.addConstrs((w >= x[i][1] for i in orders), 'makespan')
Q3_makespan.addConstr((quicksum(l[i] for i in orders) == a), 'delay num')
# Q3_makespan.addConstr((w == 6.1), 'delay num')

Q3_makespan.update()

In [33]:
Q3_makespan.optimize()

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (mac64[rosetta2])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 5878 rows, 694 columns and 28125 nonzeros
Model fingerprint: 0xb0b7e399
Variable types: 19 continuous, 675 integer (675 binary)
Coefficient statistics:
  Matrix range     [1e+00, 8e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-01, 2e+02]
Found heuristic solution: objective 14.6586914
Presolve removed 452 rows and 191 columns
Presolve time: 0.02s
Presolved: 5426 rows, 503 columns, 27346 nonzeros
Variable types: 19 continuous, 484 integer (484 binary)

Explored 0 nodes (0 simplex iterations) in 0.05 seconds (0.05 work units)
Thread count was 8 (of 8 available processors)

Solution count 1: 14.6587 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.465869140625e+01, best bound 1.465869140625e+01, gap 0.0000%


In [34]:
print("objective value =", Q3_makespan.objVal)

objective value = 14.65869140625


## Results

In [35]:
x_opt = {}
for i in orders:
    x_opt[i] = {}
    for s in stages:
        x_opt[i][s] = {}

y_opt = {}
for i in orders:
    y_opt[i] = {}
    for s in stages:
        y_opt[i][s] = {}
        for m in machines:
            y_opt[i][s][m] = {}

for var in Q3_makespan.getVars():
    index = [i for i in var.varName[2:].split(',')]
    if var.varName[0] == 'x':
        x_opt[int(index[0])][int(index[1])] = var.x
    elif var.varName[0] == 'y':
        y_opt[int(index[0])][int(index[1])][int(index[2])] = var.x

## Plot Gantt Charts

In [36]:
gantt_plot_2_3(x_opt, P, y_opt, 3)

In [37]:
#optimality_gap for huristic1_1_2
AbsoluteErrorTardy = Q3_delay.objVal - tardyamount0
if(Q3_delay.objVal == 0):
    PercentageErrorTardy = 0
else:
    PercentageErrorTardy = (Q3_delay.objVal - tardyamount0)/float(Q3_delay.objVal)
    
AbsoluteErrorMakespan = Q3_makespan.objVal - makespan0
PercentageErrorMakespan = (Q3_makespan.objVal - makespan0)/float(Q3_makespan.objVal)
print('Absolute_Error_Tardy =', AbsoluteErrorTardy)
print('Percentage_Error_Tardy =', PercentageErrorTardy)
print('Absolute_Error_Makespan =', AbsoluteErrorMakespan)
print('Percentage_Error_Makespan =', PercentageErrorMakespan)

Absolute_Error_Tardy = 0.0
Percentage_Error_Tardy = 0
Absolute_Error_Makespan = -8.391002737084818e-08
Percentage_Error_Makespan = -5.724250892891545e-09


In [38]:
#optimality_gap for huristic_paper
AbsoluteErrorTardy = Q3_delay.objVal - tardyamount
if(Q3_delay.objVal == 0):
    PercentageErrorTardy = 0
else:
    PercentageErrorTardy = (Q3_delay.objVal - tardyamount)/float(Q3_delay.objVal)
    
AbsoluteErrorMakespan = Q3_makespan.objVal - makespan
PercentageErrorMakespan = (Q3_makespan.objVal - makespan)/float(Q3_makespan.objVal)
print('Absolute_Error_Tardy =', AbsoluteErrorTardy)
print('Percentage_Error_Tardy =', PercentageErrorTardy)
print('Absolute_Error_Makespan =', AbsoluteErrorMakespan)
print('Percentage_Error_Makespan =', PercentageErrorMakespan)

Absolute_Error_Tardy = 0.0
Percentage_Error_Tardy = 0
Absolute_Error_Makespan = -8.391002737084818e-08
Percentage_Error_Makespan = -5.724250892891545e-09
